In [8]:
!pwd

/Users/ayyappatata/DL_P/2/PyTorch-YOLOv3


In [3]:
import os
import numpy as np
import random
import glob
import warnings
import shutil
import tqdm
from pathlib import Path
from terminaltables import AsciiTable

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.transforms as T

import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.transforms import functional as F

from pytorchyolo.models import load_model
from pytorchyolo.train import _create_data_loader
from pytorchyolo.test import _create_validation_data_loader, _evaluate

from pytorchyolo.utils.optimizer import get_optimizer
from pytorchyolo.utils.datasets import *
from pytorchyolo.utils.logger import Logger
from pytorchyolo.utils.loss import compute_loss
from pytorchyolo.utils.parse_config import parse_data_config
from pytorchyolo.utils.utils import load_classes,provide_determinism,to_cpu

In [4]:
class CFG:
    if torch.cuda.is_available():
      device = 'cuda'  
      NUM_DEVICES = torch.cuda.device_count()
    elif torch.backends.mps.is_available():
      device = 'mps'
      NUM_DEVICES = torch.mps.device_count()
    else:
      device ='cpu' 
      NUM_DEVICES = os.cpu_count()
      
    n_cpu=  os.cpu_count()
    mini_batch_size = 8
    multiscale_training = False
    logdir = "/Users/ayyappatata/DL_P/2/PyTorch-YOLOv3/logs/"
    epochs = 5
    seed = 1
    checkpoint_interval = 5
    evaluation_interval = 5
    

In [5]:
data_config = parse_data_config("./config/custom.data")
train_path = data_config["train"]
valid_path = data_config["valid"]
class_names = load_classes(data_config["names"])

In [6]:
model = torch.load("/Users/ayyappatata/DL_P/2/PyTorch-YOLOv3/fzd_model.pt")
params = [p for p in model.parameters() if p.requires_grad]

model = torch.load("fzd_model.pt")
for name,param in model.module_list[-10:-1].named_parameters():
  print(name, param.requires_grad)

2.conv_99.weight False
2.batch_norm_99.weight False
2.batch_norm_99.bias False
3.conv_100.weight False
3.batch_norm_100.weight False
3.batch_norm_100.bias False
4.conv_101.weight False
4.batch_norm_101.weight False
4.batch_norm_101.bias False
5.conv_102.weight False
5.batch_norm_102.weight False
5.batch_norm_102.bias False
6.conv_103.weight True
6.batch_norm_103.weight True
6.batch_norm_103.bias True
7.conv_104.weight True
7.batch_norm_104.weight True
7.batch_norm_104.bias True
8.conv_105.weight True
8.conv_105.bias True


/var/folders/4y/b_h0g7n103345tnskjhk1_s00000gn/T/ipykernel_68228/1177576970.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/Users/ayyappatata/DL_P/2

In [7]:
dataloader = _create_data_loader(
    img_path=train_path,
    img_size=model.hyperparams['height'],
    batch_size=CFG.mini_batch_size,
    n_cpu = CFG.n_cpu)

validation_dataloader = _create_validation_data_loader(
    img_path=valid_path,
    batch_size=CFG.mini_batch_size,
    img_size=model.hyperparams['height'],
    n_cpu = CFG.n_cpu)

In [8]:
def train(
    model = model,
    dataloader = dataloader,
    validation_dataloader = validation_dataloader,
    iou_thres = 0.5,
    conf_thres = 0.1,
    nms_thres = 0.5,
    class_names = class_names,
    device = CFG.device, 
    verbose = True,
    epochs= CFG.epochs,
    optimizer = "adam",
    learning_rate = 0.0001,
    weight_decay = 0.1,
    momentum = 0.9,
    logdir = CFG.logdir,
    seed = 1,
    checkpoint_interval = CFG.checkpoint_interval,
    evaluation_interval = CFG.evaluation_interval,

    ):
    
    if seed != -1:
        provide_determinism(CFG.seed)
        
    logger = Logger(logdir)
    
    optimizer = get_optimizer(model=model, 
                              optimizer_type=optimizer,
                              learning_rate=learning_rate,
                              weight_decay=weight_decay,
                              momentum=momentum
                              )
    

    for epoch in range(1, epochs+1):

            print("\n---- Training Model ----")

            model.train()  # Set model to training mode

            for batch_i, (_, imgs, targets) in enumerate(tqdm.tqdm(dataloader, desc=f"Training Epoch {epoch}")):
                batches_done = len(dataloader) * epoch + batch_i

                imgs = imgs.to(device= CFG.device, non_blocking=True)
                targets = targets.to(device = CFG.device)

                outputs = model(imgs)

                loss, loss_components = compute_loss(outputs, targets, model)

                loss.backward()

                ###############
                # Run optimizer
                ###############

                if batches_done % model.hyperparams['subdivisions'] == 0:
                    # Adapt learning rate
                    # Get learning rate defined in cfg
                    lr = model.hyperparams['learning_rate']
                    if batches_done < model.hyperparams['burn_in']:
                        # Burn in
                        lr *= (batches_done / model.hyperparams['burn_in'])
                    else:
                        # Set and parse the learning rate to the steps defined in the cfg
                        for threshold, value in model.hyperparams['lr_steps']:
                            if batches_done > threshold:
                                lr *= value
                    # Log the learning rate
                    logger.scalar_summary("train/learning_rate", lr, batches_done)
                    # Set learning rate
                    for g in optimizer.param_groups:
                        g['lr'] = lr

                    # Run optimizer
                    optimizer.step()
                    # Reset gradients
                    optimizer.zero_grad()

                # ############
                # Log progress
                # ############
                if verbose:
                    print(AsciiTable(
                        [
                            ["Type", "Value"],
                            ["IoU loss", float(loss_components[0])],
                            ["Object loss", float(loss_components[1])],
                            ["Class loss", float(loss_components[2])],
                            ["Loss", float(loss_components[3])],
                            ["Batch loss", to_cpu(loss).item()],
                        ]).table)

                # Tensorboard logging
                tensorboard_log = [
                    ("train/iou_loss", float(loss_components[0])),
                    ("train/obj_loss", float(loss_components[1])),
                    ("train/class_loss", float(loss_components[2])),
                    ("train/loss", to_cpu(loss).item())]
                logger.list_of_scalars_summary(tensorboard_log, batches_done)

                model.seen += imgs.size(0)

            # #############
            # Save progress
            # #############

            # Save model to checkpoint file
            if epoch % checkpoint_interval == 0:
                checkpoint_path = f"checkpoints/yv3_cpt_{epoch}.pt"
                print(f"---- Saving checkpoint to: '{checkpoint_path}' ----")
                torch.save(model, checkpoint_path)

            # ########
            # Evaluate
            # ########

            if epoch % evaluation_interval == 0:
                print("\n---- Evaluating Model ----")
                # Evaluate the model on the validation set
                metrics_output = _evaluate(
                    model,
                    validation_dataloader,
                    class_names,
                    img_size=model.hyperparams['height'],
                    iou_thres=iou_thres,
                    conf_thres=conf_thres,
                    nms_thres=nms_thres,
                    verbose=verbose
                )

                if metrics_output is not None:
                    precision, recall, AP, f1, ap_class = metrics_output
                    evaluation_metrics = [
                        ("validation/precision", precision.mean()),
                        ("validation/recall", recall.mean()),
                        ("validation/mAP", AP.mean()),
                        ("validation/f1", f1.mean())]
                    logger.list_of_scalars_summary(evaluation_metrics, epoch)


    if __name__ == "__main__":
        train()

In [9]:
train(
    model = model,
    dataloader = dataloader,
    validation_dataloader = validation_dataloader,
    iou_thres = 0.5,
    conf_thres = 0.1,
    nms_thres = 0.5,
    class_names = class_names,
    device = CFG.device, 
    verbose = True,
    epochs= CFG.epochs,
    optimizer = "adam",
    learning_rate = 0.0001,
    weight_decay = 0.1,
    momentum = 0.9,
    logdir = CFG.logdir,
    seed = 1,
    checkpoint_interval = CFG.checkpoint_interval,
    evaluation_interval = CFG.evaluation_interval
    )


---- Training Model ----


Training Epoch 1:   0%|          | 0/37 [00:03<?, ?it/s]

Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
Could not apply transform.
C

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/anaconda3/envs/tf/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/opt/anaconda3/envs/tf/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
  File "/Users/ayyappatata/DL_P/2/PyTorch-YOLOv3/pytorchyolo/utils/datasets.py", line 145, in collate_fn
    paths, imgs, bb_targets = list(zip(*batch))
ValueError: not enough values to unpack (expected 3, got 0)
